### PyImageJ Testing 
Testing PyImageJ's capabilities to do particle image processing

In [2]:
import imagej
from imagej import Mode
import cv2
import pandas as pd

Initializing ImageJ 

In [2]:
# initalize imagej
ij = imagej.init('net.imagej:imagej+net.imagej:imagej-legacy', mode=Mode.HEADLESS)

Opening Image

In [3]:
# macro command 
input_image = '/Users/sarah/Documents/areospace research/material recycling project/PyImageJ-Particle-Analysis/Particle Images/Ti64_Lot232-EZ2316_1Use_10X_Scale.png'
output_image = '/Users/sarah/Documents/areospace research/material recycling project/PyImageJ-Particle-Analysis/Processed Testing Output/processed_img.png'
results_summary = '/Users/sarah/Documents/areospace research/material recycling project/PyImageJ-Particle-Analysis/Processed Testing Output/results_summary.csv'

macro_cmd = f"""
// open image file 
open("{input_image}");

// convert image to 8-bit image type
run("8-bit");

// set threshold for image and invert
setThreshold(70, 255);
run("Invert");

// convert to a binary mask 
run("Convert to Mask");

// save preprocessed image
saveAs("png", "{output_image}");

// implement scale 
 run("Set Scale...", "distance=180 known=100 unit=um"); 

// set measurements
run("Set Measurements...", "area mean standard min centroid perimeter bounding box fit shape feret's integrated density median skewness kurtosis area_fraction stack position")

// analyze particles 
run("Analyze Particles...","size=0-Infinity exclude");

// save analyzed particle results to summary csv file
saveAs("Results", "{results_summary}");
"""

ij.py.run_macro(macro_cmd)

Operating in headless mode - the original ImageJ will have limited functionality.


<java object 'org.scijava.script.ScriptModule'>

In [14]:
output_image = '/Users/sarah/Documents/areospace research/material recycling project/PyImageJ-Particle-Analysis/Processed Images/Ti64_Lot232-EZ2316_1Use_10X_Scale_processedimage.png'
results_summary = '/Users/sarah/Documents/areospace research/material recycling project/PyImageJ-Particle-Analysis/Processed Results/Ti64_Lot232-EZ2316_1Use_10X_Scale_results.csv'
# open image
img = cv2.imread(output_image)

#create pandas df and iterativly label each particle 
particle_analysis_df = pd.read_csv(results_summary)

# get X, Y and labels for each particle 
label = particle_analysis_df.iloc[:,0]

# create list of tuples 
coordinates = particle_analysis_df.filter(['X','Y'], axis=1)


# Convert the coordinates to a list of tuples and multiply each by 100
tuples_list = [(int((row['X']*280)/100), int((row['Y']*280)/100)) for index, row in coordinates.iterrows()]


radius = 10
   
# Blue color in BGR 
color = (255, 0, 0) 

# Line thickness of 2 px 
thickness = -1

for coordinate in tuples_list: 
    cv2.circle(img, coordinate, radius, color, thickness)

processed_image_path = '/Users/sarah/Documents/areospace research/material recycling project/PyImageJ-Particle-Analysis/Processed Images/Ti64_Lot232-EZ2316_1Use_10X_Scale_results.png'
cv2.imwrite(processed_image_path, img)

True